In [0]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split ,col
from pyspark.sql.functions import unix_timestamp ,col

In [0]:
filepath = '/FileStore/tables/clinicaltrial_2021.csv'
clinicalDf =spark.read.options(delimiter='|', header ='true').csv(filepath)
clinicalDf.display(10)

Id,Sponsor,Status,Start,Completion,Type,Submission,Conditions,Interventions
NCT02758028,The University of Hong Kong,Recruiting,Aug 2005,Nov 2021,Interventional,Apr 2016,null,null
NCT02751957,Duke University,Completed,Jul 2016,Jul 2020,Interventional,Apr 2016,"Autistic Disorder,Autism Spectrum Disorder",null
NCT02758483,Universidade Federal do Rio de Janeiro,Completed,Mar 2017,Jan 2018,Interventional,Apr 2016,Diabetes Mellitus,null
NCT02759848,Istanbul Medeniyet University,Completed,Jan 2012,Dec 2014,Observational,May 2016,"Tuberculosis,Lung Diseases,Pulmonary Disease",null
NCT02758860,University of Roma La Sapienza,"Active, not recruiting",Jun 2016,Sep 2020,Observational [Patient Registry],Apr 2016,"Diverticular Diseases,Diverticulum,Diverticulosis",null
NCT02757209,Consorzio Futuro in Ricerca,Completed,Apr 2016,Jan 2018,Interventional,Apr 2016,Asthma,"Fluticasone,Xhance,Budesonide,Formoterol Fumarate,Salmeterol Xinafoate"
NCT02752438,Ankara University,Unknown status,May 2016,Jul 2017,Observational [Patient Registry],Apr 2016,Hypoventilation,null
NCT02753543,Ruijin Hospital,Unknown status,Nov 2015,Nov 2019,Interventional,Apr 2016,Lymphoma,null
NCT02757508,Washington University School of Medicine,Completed,Mar 2016,Jul 2017,Interventional,Apr 2016,null,Vitamins
NCT02753530,Orphazyme,Completed,Aug 2017,Jan 2021,Interventional,Apr 2016,Myositis,null


In [0]:
#Question 1
clinicalDf.count()

Out[3]: 387261

In [0]:
#Question 2
typeDF = clinicalDf.groupBy('Type').count()
typeDF.sort('count', ascending = False).display()

Type,count
Interventional,301472
Observational,77540
Observational [Patient Registry],8180
Expanded Access,69


In [0]:
#Questions 3
explodeDf = clinicalDf.withColumn('Conditions',explode(split(clinicalDf['conditions'],',')))
condDF = explodeDf.select('conditions')
sorted_condDF = condDF.groupBy('Conditions').count()
sorted_condDF.sort('count', ascending = False).na.drop().limit(5).display()

Conditions,count
Carcinoma,13389
Diabetes Mellitus,11080
Neoplasms,9371
Breast Neoplasms,8640
Syndrome,8032


In [0]:
#Question 4
meshDF = spark.read.options(delimiter=',', header ='true').csv('/FileStore/tables/mesh.csv')
rootDF =  meshDF.withColumn('roots',col('tree').substr(1, 3))
joinedDF = condDF.join(rootDF,condDF.conditions == rootDF.term,'inner')
condition_rootDF = joinedDF.groupBy('roots').count()
condition_rootDF.sort('count', ascending = False).limit(5).display()

roots,count
C04,143994
C23,136079
C01,106674
C14,94523
C10,92310


In [0]:
#Question 5
pharmaDF = spark.read.options(delimiter=',', header ='true').csv('/FileStore/tables/pharma.csv')
parentcoy_DF = pharmaDF.select('Parent_Company')

In [0]:
#Question 5
sponsordf = clinicalDf.select('Sponsor')
sponsor_pharmadf = sponsordf.join(parentcoy_DF,sponsordf.Sponsor == parentcoy_DF.Parent_Company, 'leftanti') 
nopharma_sponsorDF = sponsor_pharmadf.groupBy('Sponsor').count()
nopharma_sponsorDF.sort('count', ascending = False).limit(10).display()

Sponsor,count
National Cancer Institute (NCI),3218
M.D. Anderson Cancer Center,2414
Assistance Publique - Hôpitaux de Paris,2369
Mayo Clinic,2300
Merck Sharp & Dohme Corp.,2243
Assiut University,2154
Novartis Pharmaceuticals,2088
Massachusetts General Hospital,1971
Cairo University,1928
Hoffmann-La Roche,1828


In [0]:
#Question 6
completeddf = clinicalDf.select('Status','Completion')
completeddf_status = completeddf.filter(completeddf.Status=='Completed')
completeddf_2021 =completeddf_status.filter(completeddf.Completion.contains('2021'))
completeddf_month = completeddf_2021.select('completion').withColumn('month', col('completion').substr(1, 3))
completeddf_sorted = completeddf_month.groupBy('month').count()
completeddf_sorted.sort(unix_timestamp(col('month'),'MMM')).display()

month,count
Jan,1131
Feb,934
Mar,1227
Apr,967
May,984
Jun,1094
Jul,819
Aug,700
Sep,528
Oct,187
